In [1]:
import string
import random
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def extract_flipkart_comments(code,product_url):
    # Send a GET request to the product URL
    df1=pd.DataFrame({})  
    if product_url:
        print(product_url)
    else:
        return df1,0
    response = requests.get(product_url)
    time.sleep(0.5)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
             
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all the elements containing comments
        comment_elements = soup.find_all('div', class_='col _2wzgFH K0kLPL _1QgsS5')
        
        # Extract comments and ratings
        #df1=pd.DataFrame({})
        if len(comment_elements)==0:
            print('empty page',end=' ')
        else:
            for element in comment_elements:
                df = {}
                #print(element)
                df['code']=[code]
                df['customer']=[element.find('p').text.strip()]
                rating=element.find('div', class_='_3LWZlK _1BLPMq _3B8WaH')
                if rating:
                    df['rating'] = [rating.text.strip()]
                df['comment'] = [element.find('div', class_='_6K-7Co').text.strip()]
                df['date']=[element.find("div",{"class":"row _1ExUpQ"}).find_all("p",{"class":"_2sc7ZR"})[1].get_text()]
                df['district']=[element.findAll('p')[-1].text.strip()]
                #print(df)
                df=pd.DataFrame(df)
                df1=pd.concat([df1,df])
        next=soup.findAll('a',{'class':'_1LKTO3'})
        #print(next)
        if next:
            if str(next[-1].text.strip())=='Next':
                return df1,'https://www.flipkart.com'+next[-1]['href']
            else:
                return df1,0
    else:
        print("Failed to retrieve data from Flipkart")
    return df1,0

In [3]:
def total(code,url):
    df,n=extract_flipkart_comments(code,url)
    while n!=0:
        #try:
            df1,n=extract_flipkart_comments(code,n)
            df=pd.concat([df,df1])
        #except
            #print('Failed to extract')

            #n=0
    #print(df)
    return df

In [4]:
df=pd.read_json('vebnor.json')
df['revb']=df.total.apply(lambda x: x[-1])
df['code']=df.total.apply(lambda x:x[1])
l=list(df.revb)
unq=list(df.code)

In [5]:
#total(l[0])
df4=pd.DataFrame({})
for i in range(len(l)):
    print(i)
    df4=pd.concat([df4,total(unq[i],l[i])])

0
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&marketplace=FLIPKART
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&marketplace=FLIPKART&page=2
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&marketplace=FLIPKART&page=3
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&marketplace=FLIPKART&page=4
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&marketplace=FLIPKART&page=5
https://www.flipkart.com/vebnor-men-solid-casual-grey-shirt/product-reviews/itme20c3d25028b9?pid=SHTFVC7FBFZPJAHD&lid=LSTSHTFVC7FBFZPJAHDWC3BAU&m

ReadTimeout: HTTPSConnectionPool(host='www.flipkart.com', port=443): Read timed out. (read timeout=None)

In [7]:
df4.to_csv('comments_vebnor.csv')

In [12]:
df4.date=df4[['customer','date']].apply(lambda x : x.date[len(x.customer):])

AttributeError: 'Series' object has no attribute 'date'

In [11]:
df4.columns

Index(['code', 'customer', 'rating', 'comment', 'date', 'district'], dtype='object')